In [3]:
import pandas as pd

df = pd.read_csv("../data/df_all_voi.csv")

print(df.columns)
print(df.head())
print(df.shape)


Index(['Identificativo noleggio', 'Targa veicolo', 'Data inizio corsa',
       'Data fine corsa', 'Lat inizio corsa_coordinate',
       'Lon inizio corsa_coordinate', 'Lat fine corsa_coordinate',
       'Lon fine corsa_coordinate', 'Tempo Tot', 'KM Tot', 'Prenotazione',
       'Batteria inizio', 'Batteria fine', 'source_file', 'start_hour',
       'start_date', 'weekday', 'is_weekend', 'duration_hours', 'speed_kmh',
       'trip_date', 'year', 'hour', 'month', 'week'],
      dtype='str')
                Identificativo noleggio Targa veicolo    Data inizio corsa  \
0  6dd0c93b-4557-4e8d-8f98-a543f24a761d          besc  2024-10-31 22:59:26   
1  201d1181-ad9a-4722-8754-280166181dbe          j2b6  2024-10-31 22:51:43   
2  201d1181-ad9a-4722-8754-280166181dbe          j2b6  2024-10-31 22:51:43   
3  9ac9534b-b86a-4c78-b5b0-39c9e3171227          yvld  2024-10-31 22:50:08   
4  d1caf896-84d3-4cb6-9d4a-03e538658dcc          9qxg  2024-10-31 22:47:52   

       Data fine corsa  Lat inizio cor

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv("../data/df_all_voi.csv")

# Rename important columns to simple names
df = df.rename(columns={
    "Data inizio corsa": "start_time",
    "Data fine corsa": "end_time",
    "Lat inizio corsa_coordinate": "start_lat",
    "Lon inizio corsa_coordinate": "start_lon",
    "Lat fine corsa_coordinate": "end_lat",
    "Lon fine corsa_coordinate": "end_lon",
    "Tempo Tot": "duration_raw",
    "KM Tot": "distance_km"
})

# Convert timestamps
df["start_time"] = pd.to_datetime(df["start_time"])
df["end_time"] = pd.to_datetime(df["end_time"])

# Duration in minutes (more reliable than raw field)
df["duration_min"] = (df["end_time"] - df["start_time"]).dt.total_seconds() / 60

# Remove bad data
before_clean = len(df)

df = df[
    (df["duration_min"] > 0) &
    (df["start_lat"].notna()) &
    (df["end_lat"].notna())
]

after_clean = len(df)

print("Records before cleaning:", before_clean)
print("Records after cleaning:", after_clean)

# Extract time variables
df["hour"] = df["start_time"].dt.hour
df["weekday"] = df["start_time"].dt.weekday
df["month"] = df["start_time"].dt.month

# Define peak periods
def peak_label(hour):
    if 7 <= hour <= 10:
        return "Morning Peak"
    elif 17 <= hour <= 20:
        return "Evening Peak"
    else:
        return "Off Peak"

df["period"] = df["hour"].apply(peak_label)

print(df.head())


Records before cleaning: 291148
Records after cleaning: 285025
                Identificativo noleggio Targa veicolo          start_time  \
0  6dd0c93b-4557-4e8d-8f98-a543f24a761d          besc 2024-10-31 22:59:26   
1  201d1181-ad9a-4722-8754-280166181dbe          j2b6 2024-10-31 22:51:43   
2  201d1181-ad9a-4722-8754-280166181dbe          j2b6 2024-10-31 22:51:43   
3  9ac9534b-b86a-4c78-b5b0-39c9e3171227          yvld 2024-10-31 22:50:08   
4  d1caf896-84d3-4cb6-9d4a-03e538658dcc          9qxg 2024-10-31 22:47:52   

             end_time  start_lat  start_lon    end_lat   end_lon  \
0 2024-10-31 23:01:19  45.043269   7.671645  45.048233  7.673718   
1 2024-10-31 22:59:10  45.074264   7.694650  45.086248  7.704112   
2 2024-10-31 22:59:10  45.074264   7.694650  45.086248  7.704112   
3 2024-10-31 23:04:01  45.076192   7.696033  45.086265  7.704420   
4 2024-10-31 23:03:58  45.074614   7.614285  45.058371  7.658567   

   duration_raw  distance_km  ... is_weekend  duration_hours  spe

In [6]:
df0 = pd.read_csv("../data/df_all_voi.csv")

print("Initial records:", len(df0))

# --- Rename ---
df0 = df0.rename(columns={
    "Data inizio corsa": "start_time",
    "Data fine corsa": "end_time",
    "Lat inizio corsa_coordinate": "start_lat",
    "Lon inizio corsa_coordinate": "start_lon",
    "Lat fine corsa_coordinate": "end_lat",
    "Lon fine corsa_coordinate": "end_lon",
    "Tempo Tot": "duration_raw",
    "KM Tot": "distance_km"
})

# --- Timestamp conversion ---
df0["start_time"] = pd.to_datetime(df0["start_time"], errors="coerce")
df0["end_time"] = pd.to_datetime(df0["end_time"], errors="coerce")

before_timestamp = len(df0)
df1 = df0.dropna(subset=["start_time", "end_time"])
after_timestamp = len(df1)

print("Removed due to invalid timestamps:", before_timestamp - after_timestamp)

# --- Duration calculation ---
df1["duration_min"] = (df1["end_time"] - df1["start_time"]).dt.total_seconds() / 60

before_duration = len(df1)
df2 = df1[df1["duration_min"] > 0]
after_duration = len(df2)

print("Removed due to non-positive duration:", before_duration - after_duration)

# --- Coordinate cleaning ---
before_coord = len(df2)
df3 = df2.dropna(subset=["start_lat", "start_lon", "end_lat", "end_lon"])
after_coord = len(df3)

print("Removed due to missing coordinates:", before_coord - after_coord)

print("Final cleaned records:", after_coord)


Initial records: 291148
Removed due to invalid timestamps: 6123
Removed due to non-positive duration: 0
Removed due to missing coordinates: 0
Final cleaned records: 285025


In [7]:
import pandas as pd

df_raw = pd.read_csv("../data/df_all_voi.csv")

# rename like before
df_raw = df_raw.rename(columns={
    "Data inizio corsa": "start_time",
    "Data fine corsa": "end_time"
})

# convert with coercion
df_raw["start_time_parsed"] = pd.to_datetime(df_raw["start_time"], errors="coerce")
df_raw["end_time_parsed"] = pd.to_datetime(df_raw["end_time"], errors="coerce")

# find invalid timestamps
invalid_time_rows = df_raw[
    df_raw["start_time_parsed"].isna() |
    df_raw["end_time_parsed"].isna()
]

print("Total invalid timestamp rows:", len(invalid_time_rows))

# show 10 examples
invalid_time_rows[[
    "start_time",
    "end_time"
]].head(10)


Total invalid timestamp rows: 6123


,start_time,end_time
117815,NaN,NaN
117816,NaN,NaN
117817,NaN,NaN
117818,NaN,NaN
117819,NaN,NaN
117820,NaN,NaN
117821,NaN,NaN
117822,NaN,NaN
117823,NaN,NaN
117824,NaN,NaN
